In [15]:
from snowflake.snowpark.session import Session

In [16]:
CONNECTION_PARAMETERS = {
    "account": "ug94937.us-east4.gcp",
    "user":"ADITYASINGH",
    "password": os.environ.get('SF_Password'),
    "role": "ADITYASINGH",
    "database": "FIRST_DB",
    "warehouse": "FOSFOR_INSIGHT_WH",
    "schema": "PUBLIC"
}

In [22]:
import uuid
# project_id = str(uuid.uuid4())
run_id = str(uuid.uuid4())
print(run_id)

52dfff45-6981-40e0-9f91-d0e6a7a791b7


In [18]:
# payload-1 (classification Airline Delay dataset)
exp_data = '''{{
"source":"EXP", 
"project_id":"0e0fb803-22db-4d88-9f2f-f6f75b6abcf0", 
"id":"7bbb5061-54d4-4862-8d47-7fbee388a4d1", 
"run_id":"{0}", 
"exp_name": "Final_recipe", 
"algorithm_type":"classification", 
"algo_details": {{"snowflake.ml.modeling.xgboost.XGBClassifier": null}}, 
"dataset": "AIRLINE_DEP_DELAY_10K", 
"target_column": "DEP_DEL15"}}'''.format(run_id)

In [ ]:
# payload-2 (Regression Alcohol dataset)
exp_data = f'''{{
"source":"EXP", 
"project_id":"0e0fb803-22db-4d88-9f2f-f6f75b6abcf0", 
"id":"7bbb5061-54d4-4862-8d47-7fbee388a4d1", 
"run_id":"{0}", 
"exp_name": "Final_recipe", 
"algorithm_type":"regression", 
"algo_details": {{"snowflake.ml.modeling.ensemble.RandomForestRegressor": null}}, 
"dataset": "ALCOHOL_DATA_10L", 
"target_column": "QUALITY"}}'''.format(run_id)

In [ ]:
# payload-3 (classification Employee dataset)
exp_data = '''{{
"source":"EXP", 
"project_id":"0e0fb803-22db-4d88-9f2f-f6f75b6abcf0", 
"id":"7bbb5061-54d4-4862-8d47-7fbee388a4d1", 
"run_id":"{0}", 
"exp_name": "Final_recipe", 
"algorithm_type":"classification", 
"algo_details": {{"snowflake.ml.modeling.linear_model.LogisticRegression": null}}, 
"dataset": "EMPLOYEE_10L", 
"target_column": "LEAVEORNOT"}}'''.format(run_id)

In [27]:
# payload-4 (Regression Diamonds dataset)
exp_data = '''{{
"source":"EXP", 
"project_id":"0e0fb803-22db-4d88-9f2f-f6f75b6abcf0", 
"id":"7bbb5061-54d4-4862-8d47-7fbee388a4d1", 
"run_id":"{0}", 
"exp_name": "Final_recipe", 
"algorithm_type":"regression", 
"algo_details": {{"snowflake.ml.modeling.ensemble.RandomForestRegressor": null}}, 
"dataset": "DIAMONDS", 
"target_column": "PRICE"}}'''.format(run_id)

In [28]:
exp_data

'{\n"source":"EXP", \n"project_id":"0e0fb803-22db-4d88-9f2f-f6f75b6abcf0", \n"id":"7bbb5061-54d4-4862-8d47-7fbee388a4d1", \n"run_id":"52dfff45-6981-40e0-9f91-d0e6a7a791b7", \n"exp_name": "Final_recipe", \n"algorithm_type":"regression", \n"algo_details": {"snowflake.ml.modeling.ensemble.RandomForestRegressor": null}, \n"dataset": "DIAMONDS", \n"target_column": "PRICE"}'

In [35]:
def create_stage(session, stage_name="demo"):
    try:
        session.sql(f"create or replace stage {stage_name}").collect()
        return f"@{stage_name}"
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex

        
def get_session():
    """
    Method creates snowflake session object.
    :return:
    """
    try:
        return Session.builder.configs(CONNECTION_PARAMETERS).create()
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex


# Stored Procedure 
def run_experiment(session: Session, exp_data: str) -> list:
    # Imports
    from snowflake.ml.modeling.pipeline import Pipeline
    from snowflake.ml.modeling.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
    from snowflake.ml.modeling.metrics import r2_score, accuracy_score, precision_score, roc_auc_score, \
        f1_score, recall_score, log_loss, mean_absolute_error, mean_squared_error
    from snowflake.snowpark.functions import col, is_null, regexp_replace, when, lit
    from snowflake.snowpark.types import StringType
    from snowflake.snowpark.exceptions import SnowparkSQLException
    import importlib, sys, json, os, logging
    from snowflake.ml.registry.registry import Registry
    
    
    # Functions used in stored proc
    def apply_data_cleansing(df):
        """
        Method handles null values in snowpark dataframe.
        :param:
        df: input dataframe
        :returns:
        df_cleaned: dataframe after null handling
        """
        # fillna - Unknown and 0
        schema_fields = df.schema.fields
        fill_values = {field.name: "UNKNOWN" if isinstance(field.datatype, StringType) else 0 for field in schema_fields}
        df_cleaned = df.fillna(fill_values)
        return df_cleaned


    def get_feature_columns(df):
        """
        Identifies the numerical and categorical features in dataset.
        Identifies features for label encoding and one hot encoding
        :param:
        df: input dataframe
        :returns:
        categorical_features: list of non-numerical feature columns
        numerical_features: list of numerical feature columns
        le_column_features: list of feature label encoder columns
        oh_column_features: list of feature one hot encoder columns
        """
        schema_fields = df.schema.fields
        features = df.columns
        features.remove(exp_details.get("target_column"))
        df_schema = session.sql(f"DESCRIBE TABLE {exp_details.get('dataset')}").collect()
        categorical_types = ['VARCHAR','CHAR','STRING','TEXT','BOOL']
        categorical_features = []  
        for row in df_schema:
            for typ in categorical_types:
                if typ in row['type']:
                    categorical_features.append(row['name'])
                    break
        numerical_features = list(set(features) - set(categorical_features))
        print(f"numerical_features:  {numerical_features}")
        print(f"categorical_features: {categorical_features}")
        
        #identify columns for labelencoding and onehotencoding   
        le_column_features = []
        oh_column_features = []
        if len(categorical_features) >= 1:
            print(f"{categorical_features} columns are non numeric in feature dataset, encoding required.")
            for column in categorical_features:
                if df.select(df[column]).distinct().count() >= 10:
                    le_column_features.append(column)
                elif column == exp_details.get("target_column"):
                    le_column_feature.append(column)
                else:
                    oh_column_features.append(column)
            print(f"Columns identified to be encoded with label encoder: {le_column_features}")
            print(f"Columns identified to be encoded with one hot encoder: {oh_column_features}")
        return categorical_features, numerical_features, le_column_features, oh_column_features


    def create_and_run_preprocessing(df, categorical_features, numerical_features, le_column_features, oh_column_features):
        """
        Based on different features column input creates preprocessing steps and run it on input dataframe
        :param:
        df: input dataframe
        categorical_features: list of non-numerical feature columns
        numerical_features: list of numerical feature columns
        le_column_features: list of feature label encoder columns
        oh_column_features: list of feature one hot encoder columns
        :returns:
        df_train: preprocessed train dataset
        df_test: preprocessed test dataset
        """
        #pipeline steps 
        print("Setting up preprocessing pipeline based on dataset")
        categorical_pp = {f'le_{column}':LabelEncoder(input_cols=column, output_cols=column) for column in le_column_features}
        if len(oh_column_features)>0:
            categorical_pp['oh_enc'] = OneHotEncoder(input_cols=oh_column_features, output_cols=oh_column_features, handle_unknown='ignore')
        numerical_pp = {
            'scaler': MinMaxScaler(input_cols=numerical_features, output_cols=numerical_features)
        }
        steps = [(key, categorical_pp[key]) for key in categorical_pp if categorical_pp[key]!=[]] + \
        [(key, numerical_pp[key]) for key in numerical_pp if numerical_features!=[]]
        print(f"Selected preprocesing steps: \n{steps}")    
            
        # Run preprocessing pipeline steps 
        print("Running data preprocessing pipeline")
        df = Pipeline(steps=steps).fit(df).transform(df)
        print(f"Transformed dataset: \n {df.show()}")
        df_train, df_test = df.random_split(weights=[0.8, 0.2], seed=0)
        return df_train, df_test


    def run_estimator(df_train, df_test, input_cols):
        """
        trains on df_train and creates model object for given algorithm/estimator.
        runs prediction function of model object on test dataset
        :param:
        df_train: input training dataframe
        df_test: input test dataframe
        input_cols: list of input feature names
        :returns:
        df_pred: output predict dataframe
        """
        for algorithm, hyperparam in exp_details.get("algo_details").items():
            algorithm = algorithm.rsplit('.', 1)
            module = importlib.import_module(algorithm[0])
            print(f"----Running Algorithm {algorithm[1]}----")
            attr = getattr(module, algorithm[1])
            pipe = Pipeline(steps=[("algorithm", attr(input_cols=input_cols
                                                  , label_cols=[exp_details.get("target_column")]
                                                  , output_cols=['PREDICTIONS']))]
                   )
    
            # Fit the pipeline
            print(f"Running model pipeline {algorithm[1]}")
            model = pipe.fit(df_train)
     
            # Test the model
            print("Running prediction on model with test dataset")
            df_pred = model.predict(df_test)
            return model, df_pred

    
    def try_or(fn):
        try:
            out = fn()
            return out
        except:
            return None
    
        
    def eval_metrics(df_pred):
        print("Generating Metrices")
        if exp_details.get("algorithm_type") == 'classification':
            return {
            'accuracy_score': try_or(lambda: accuracy_score(df=df_pred, y_true_col_names=exp_details.get("target_column"), y_pred_col_names='PREDICTIONS')),
            'f1_score': try_or(lambda: f1_score(df=df_pred, y_true_col_names=exp_details.get("target_column"), y_pred_col_names='PREDICTIONS')),
            'recall_score': try_or(lambda: recall_score(df=df_pred, y_true_col_names=exp_details.get("target_column"), y_pred_col_names='PREDICTIONS')),
            'precision_score': try_or(lambda: precision_score(df=df_pred, y_true_col_names=exp_details.get("target_column"), y_pred_col_names='PREDICTIONS')),
            'roc_auc': try_or(lambda: roc_auc_score(df=df_pred, y_true_col_names=exp_details.get("target_column"), y_score_col_names='PREDICTIONS')),
            'log_loss': try_or(lambda: log_loss(df=df_pred, y_true_col_names=exp_details.get("target_column"), y_pred_col_names='PREDICTIONS'))
            }
        else:
            return {
            'r2_score': try_or(lambda: r2_score(df=df_pred, y_true_col_name=exp_details.get("target_column"), y_pred_col_name='PREDICTIONS')),
            'mean_absolute_error': try_or(lambda: mean_absolute_error(df=df_pred, y_true_col_names=exp_details.get("target_column"), y_pred_col_names='PREDICTIONS')),
            'mean_squared_error': try_or(lambda: mean_squared_error(df=df_pred, y_true_col_names=exp_details.get("target_column"), y_pred_col_names='PREDICTIONS'))
            }


    def register_model(model, metrics_info):
        print("Started: Registering model on snowflake")
        reg = Registry(session=session)
        try:
            clean = lambda x : x.replace("-","_")
            project_id = clean(exp_details.get("project_id"))
            exp_id = clean(exp_details.get("id"))
            run_id = clean(exp_details.get("run_id"))
            model_name = exp_details.get("exp_name")+"_"+project_id+"_"+exp_id+"_"+run_id
            mv = reg.log_model(model=model,
                               model_name=model_name,
                               comment=exp_details.get("description"),
                               version_name="V1",
                               python_version="3.9.19",
                               conda_dependencies=["xgboost","scikit-learn==1.2.2"],
                               metrics=[{"model_metrics": metrics_info, 
                                         "project_id": exp_details.get("project_id"),
                                         "id": exp_details.get("id"),
                                         "run_id": exp_details.get("run_id"),
                                         "algorithm_type": exp_details.get("algorithm_type"),
                                         "algo_details": exp_details.get("algo_details"),
                                         "dataset": exp_details.get("dataset"),
                                         "target_column": exp_details.get("target_column"),
                                         "exp_name": exp_details.get("exp_name"),
                                         "source": "EXP"}])
            print("Registeration of model completed!!!")
            return model_name
        except Exception as ex:
            key = 'Processing aborted due to error 370001' 
            if key in str(ex):
                print("Registeration of model completed!!!")
                return model_name
            else:
                print("Exception Occured while registering model")
                return str(ex).split('?')
            
            
    def create_tags(session, exp_details):
        for key in exp_details.keys():
            tag = session.sql(f"CREATE TAG IF NOT EXISTS {key}")
            tag.show()
        if exp_details.get('algorithm_type')=='classification':
            metric_names=["accuracy_score","precision_score","recall_score","f1_score","roc_auc","log_loss"]
        else:
            metric_names=["r2_score","mean_absolute_error","mean_squared_error"]
        for name in metric_names:
            tag = session.sql(f"CREATE TAG IF NOT EXISTS {name}")
            tag.show()

        
    def set_tags(session, m_name, exp_details, metric_info):
        for key, value in exp_details.items():
            value = str(value).replace("'","\"")
            tag = session.sql(f"ALTER MODEL IF EXISTS {m_name} SET TAG {key}='{value}'")
            tag.show()
        for key, value in metric_info.items():
            value = str(value).replace("'","\"")
            tag = session.sql(f"ALTER MODEL IF EXISTS {m_name} SET TAG {key}='{value}'")
            tag.show()
    
    # loading experiment details
    exp_details=json.loads(exp_data)
    
    metric_names=["accuracy_score","precision_score","recall_score","f1_score","roc_auc","log_loss",""]
    
    # creating user tags if not exist
    create_tags(session, exp_details)
    
    # Reading dataset
    print("Reading dataset features")
    data = session.table(exp_details.get("dataset"))
    
    # fillna
    data = apply_data_cleansing(data)
    
    # Identify feature columns
    categorical_features, numerical_features, le_column_features, oh_column_features = get_feature_columns(data)
    
    # Based on feature, do preprocessing
    data_train, data_test = create_and_run_preprocessing(data, categorical_features, numerical_features, le_column_features, oh_column_features)
    
    # Run model training and prediction
    input_cols = data_train.columns
    input_cols.remove(exp_details.get("target_column"))    
#     input_cols = categorical_features+numerical_features
#     if exp_details.get("target_column") in categorical_features:
#         input_cols.remove(exp_details.get("target_column"))
    model, data_pred = run_estimator(data_train, data_test, input_cols)
    
    # Evaluate model metrices
    metrics_info = eval_metrics(data_pred)
    print(metrics_info)
    
    # Register model on snowflake registry
    model_name = register_model(model, metrics_info)
    print(model_name)
    
    # Set relevant tags to model object
    set_tags(session, model_name, exp_details, metrics_info)
    
    return [{"exp_name":exp_details.get("exp_name", "sample_experiment"),
             "version":"V1",
             "matrices":{"model_metrics": metrics_info, "project_id": "0001", "source": "EXP"},
             "algorithm_type":exp_details.get("algorithm_type"),
             "algorithm": list(exp_details.get("algo_details").keys()),
             "RUN_STATUS":"SUCCESS",
             "registry_model_name":exp_details.get("exp_name")+"_"+exp_details.get("project_id")+"_"+exp_details.get("id")+"_"+exp_details.get("run_id")}]

In [36]:
%%time
# Initilization
# logging.basicConfig(stream=sys.stdout, level=logging.INFO)
print("Creating Snowflake Session object...")
session = get_session()
stage = create_stage(session)
print("Session has been created !")

print("Creating stored procedure...")
session.sproc.register(func=run_experiment,
                       name="run_experiment",
                       packages=["snowflake-snowpark-python", "snowflake-ml-python"],
                       isPermanant=False,
                       stage_location=stage,
                       replace=True)
print("Stored procedure has been created successfully!")

# tagging session
print("Setting tag to session object: tag= ", run_id)
session.query_tag=run_id

print("Executing Procedure")
# procedure_response = session.call("run_experiment", exp_data)
procedure_response = run_experiment(session, exp_data)
print("Stored Procedure Executed Successfully !")
print(procedure_response)

#Log in mlflow
print("Logging in mlflow completed !")

Creating Snowflake Session object...
Session has been created !
Creating stored procedure...
Stored procedure has been created successfully!
Setting tag to session object: tag=  52dfff45-6981-40e0-9f91-d0e6a7a791b7
Executing Procedure
-----------------------------------------------
|"status"                                     |
-----------------------------------------------
|SOURCE already exists, statement succeeded.  |
-----------------------------------------------

---------------------------------------------------
|"status"                                         |
---------------------------------------------------
|PROJECT_ID already exists, statement succeeded.  |
---------------------------------------------------

-------------------------------------------
|"status"                                 |
-------------------------------------------
|ID already exists, statement succeeded.  |
-------------------------------------------

------------------------------------------

/packages/Python-3.9-Snowpark/7e0b365a-f7cb-4d2a-ba2a-e8ed02dbe50c/3.9/snowflake/ml/modeling/preprocessing/one_hot_encoder.py:845: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  state_pandas[[_CATEGORY]] = state_pandas[[_CATEGORY]].applymap(convert_to_string_excluding_nan)
/packages/Python-3.9-Snowpark/7e0b365a-f7cb-4d2a-ba2a-e8ed02dbe50c/3.9/snowflake/ml/modeling/preprocessing/one_hot_encoder.py:846: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  state_pandas[[_FITTED_CATEGORY]] = state_pandas[[_FITTED_CATEGORY]].applymap(convert_to_string_excluding_nan)
/packages/Python-3.9-Snowpark/7e0b365a-f7cb-4d2a-ba2a-e8ed02dbe50c/3.9/snowflake/ml/modeling/preprocessing/one_hot_encoder.py:845: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  state_pandas[[_CATEGORY]] = state_pandas[[_CATEGORY]].applymap(convert_to_string_excluding_nan)
/packages/Python-3.9-Snowpark/7e0b365a-f7cb-4d2a-ba2a

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"Y"                  |"DEPTH"              |"Z"                  |"X"                  |"TABLE_PCT"          |"CARAT"               |"CUT_Fair"  |"CUT_Good"  |"CUT_Ideal"  |"CUT_Premium"  |"CUT_Very Good"  |"COLOR_D"  |"COLOR_E"  |"COLOR_F"  |"COLOR_G"  |"COLOR_H"  |"COLOR_I"  |"COLOR_J"  |"CLARITY_I1"  |"CLARITY_IF"  |"CLARITY_SI1"  |"CLARITY_SI2"  |"CLARITY_VS1"  |"CLARITY_VS2"  |"CLARITY_VVS1"  |"CLARITY_VVS2"  |"CUT"      |"COLOR"  |"CLARITY"  |"PRICE"  |
----------------------------------------------------------------------

SnowparkSQLException: (1300) (1304): 01b54967-0000-6d46-0000-576d0715fcee: 100357 (P0000): Python Interpreter Error:
Traceback (most recent call last):
  File "_udf_code.py", line 77, in compute
  File "/packages/Python-3.9-Snowpark/7e0b365a-f7cb-4d2a-ba2a-e8ed02dbe50c/3.9/snowflake/ml/modeling/_internal/snowpark_implementations/snowpark_trainer.py", line 234, in fit_wrapper_function
  File "/usr/lib/python_udf/b49e96e6278f69312fc460688053a1eeb615fb489add9ace89457c0501d89752/lib/python3.9/site-packages/sklearn/base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/lib/python_udf/b49e96e6278f69312fc460688053a1eeb615fb489add9ace89457c0501d89752/lib/python3.9/site-packages/sklearn/ensemble/_forest.py", line 348, in fit
    X, y = self._validate_data(
  File "/usr/lib/python_udf/b49e96e6278f69312fc460688053a1eeb615fb489add9ace89457c0501d89752/lib/python3.9/site-packages/sklearn/base.py", line 621, in _validate_data
    X, y = check_X_y(X, y, **check_